<a href="https://colab.research.google.com/github/mraniketr/twoModeCalling/blob/master/Final_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

     |████████████████████████████████| 3.3MB 2.7MB/s 
Version:  2.2.0-rc3
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [3]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteADIPOO/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteADIPOO/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteADIPOO/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))


In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)


Epoch 1/20
30/30 [==============================] - 3s 101ms/step - loss: 0.9085 - accuracy: 0.4787 - val_loss: 0.7304 - val_accuracy: 0.5367
Epoch 2/20
30/30 [==============================] - 3s 98ms/step - loss: 0.6756 - accuracy: 0.5828 - val_loss: 0.6374 - val_accuracy: 0.6173
Epoch 3/20
30/30 [==============================] - 3s 99ms/step - loss: 0.6151 - accuracy: 0.6334 - val_loss: 0.6015 - val_accuracy: 0.6554
Epoch 4/20
30/30 [==============================] - 3s 101ms/step - loss: 0.5755 - accuracy: 0.6695 - val_loss: 0.5680 - val_accuracy: 0.6875
Epoch 5/20
30/30 [==============================] - 3s 105ms/step - loss: 0.5388 - accuracy: 0.7057 - val_loss: 0.5327 - val_accuracy: 0.7101
Epoch 6/20
30/30 [==============================] - 3s 106ms/step - loss: 0.5005 - accuracy: 0.7409 - val_loss: 0.4995 - val_accuracy: 0.7509
Epoch 7/20
30/30 [==============================] - 3s 103ms/step - loss: 0.4620 - accuracy: 0.7731 - val_loss: 0.4660 - val_accuracy: 0.7689
Epoch 8/

In [12]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3125 - accuracy: 0.8651
loss: 0.312
accuracy: 0.865


In [0]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

In [0]:
new_model = tf.keras.models.load_model('saved_model/my_model')
model=new_model

In [32]:
def emotions(predictions):
    if(predictions>=0.5):
      print("Positive")
    else:
      print("Negative")

predictions = model.predict(tf.expand_dims('Im scared', 0))
emotions(predictions)

Negative
